In [ ]:
!pip install pyarrow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import pyarrow as pa
df = pd.DataFrame({'one': [20, np.nan, 2.5],'two': ['january', 'february', 'march'],'three': [True, False, True]},index=list('abc'))
table = pa.Table.from_pandas(df)

In [ ]:
df

,one,two,three
a,20.0,january,True
b,NaN,february,False
c,2.5,march,True


In [ ]:
table

pyarrow.Table
one: double
two: string
three: bool
__index_level_0__: string
----
one: [[20,null,2.5]]
two: [["january","february","march"]]
three: [[true,false,true]]
__index_level_0__: [["a","b","c"]]

In [ ]:
table.schema

one: double
two: string
three: bool
__index_level_0__: string
-- schema metadata --
pandas: '{"index_columns": ["__index_level_0__"], "column_indexes": [{"na' + 649

In [ ]:
import requests

js=requests.get('https://api.hh.ru/vacancies?text=Разработчик').json()['items']
df1=pd.DataFrame(js)
df1

In [ ]:
table1 = pa.Table.from_pandas(df1)

In [ ]:
table1.schema

In [ ]:
table1.slice(1,5)

In [ ]:
table1.select(['id','name']).slice(0,3)

pyarrow.Table
id: string
name: string
----
id: [["48457284","71077194","70796480"]]
name: [["Разработчик Solidity/Разработчик Rust/Разработчик Go/Разработчик python(удаленно)","C++ разработчик (highload)","Java-разработчик"]]

In [ ]:
table1.flatten().shape

(20, 52)

In [ ]:
!mkdir /root/.kaggle/
!touch /root/.kaggle/kaggle.json
!echo '{"username":"vasiliykireev","key":"b3d949cd3b783dc7f6983a79797629a9"}' > /root/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d hmavrodiev/sofia-air-quality-dataset
!unzip /content/sofia-air-quality-dataset.zip -d /content/sofia/

In [ ]:
/content/sofia-air-quality-dataset.zip

In [ ]:
import pyarrow as pa
import pyarrow.csv as csv

parse_options = csv.ParseOptions(delimiter=",")
table = csv.read_csv('/content/sofia/2017-07_bme280sof.csv',  parse_options=parse_options)

table.shape

(701548, 9)

In [ ]:
dir(pa)

In [ ]:
table.schema

: int64
sensor_id: int64
location: int64
lat: double
lon: double
timestamp: timestamp[s]
pressure: double
temperature: double
humidity: double

In [ ]:
import pyarrow as pa
import pyarrow.csv as csv

table = csv.read_csv('/content/sofia/2017-07_bme280sof.csv', convert_options=csv.ConvertOptions(
    column_types={
        'pressure': pa.decimal128(precision=10, scale=2),
        'humidity': pa.decimal128(precision=10, scale=2),
    }
))

In [ ]:
table.slice(0,10)

pyarrow.Table
: int64
sensor_id: int64
location: int64
lat: double
lon: double
timestamp: timestamp[s]
pressure: decimal128(10, 2)
temperature: double
humidity: decimal128(10, 2)
----
: [[1,5,7,9,10,11,13,22,25,30]]
sensor_id: [[2266,2292,3096,3428,3472,1952,1846,3512,2228,3438]]
location: [[1140,1154,1558,1727,1750,976,923,1770,1120,1732]]
lat: [[42.738,42.663000000000004,42.7,42.623999999999995,42.669,42.708999999999996,42.64,42.683,42.693000000000005,42.738]]
lon: [[23.272,23.273000000000003,23.36,23.406,23.318,23.398000000000003,23.31,23.335,23.333000000000002,23.293000000000003]]
timestamp: [[2017-07-01 00:00:07,2017-07-01 00:00:08,2017-07-01 00:00:10,2017-07-01 00:00:12,2017-07-01 00:00:13,2017-07-01 00:00:13,2017-07-01 00:00:15,2017-07-01 00:00:24,2017-07-01 00:00:28,2017-07-01 00:00:37]]
pressure: [[95270.27,94355.83,95155.81,94679.57,94327.88,95314.52,93616.77,94962.39,94982.91,95099.81]]
temperature: [[23.46,23.06,26.53,28.34,26.31,22.66,23.87,24.92,26.29,24.62]]
humidity: [[

In [ ]:
df = pd.read_csv('/content/sofia/2017-07_bme280sof.csv')

table = pa.Table.from_pandas(df)

df_new = table.to_pandas()

schema = pa.Schema.from_pandas(df)

In [ ]:
schema

Unnamed: 0: int64
sensor_id: int64
location: int64
lat: double
lon: double
timestamp: string
pressure: double
temperature: double
humidity: double
-- schema metadata --
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 1297

Скачать данные из kaggle и создать таблицу arrow из /content/sofia/2017-07_bme280sof.csv, указав при считывании условия- 
отобрать только sensor_id,timestamp,pressure, преобразовать временную метку во датувремя, преобразовать тип данных давление в плавающую точку с точностью 3 знака после запятой
Вывести срез таблицы из первых 10 строк и колонки давление

In [ ]:
PO=csv.ParseOptions(delimiter =',')
CO=csv.ConvertOptions(include_columns=['sensor_id','timestamp','pressure'], 
    column_types={
        'pressure': pa.decimal128(precision=10, scale=3),
    })
table=csv.read_csv('/content/sofia/2017-07_bme280sof.csv',convert_options=CO,parse_options=PO)
table.schema

sensor_id: int64
timestamp: timestamp[s]
pressure: decimal128(10, 3)

In [ ]:
table.shape

(701548, 3)

In [ ]:
table.select(['pressure']).slice(5)

pyarrow.Table
pressure: decimal128(10, 3)
----
pressure: [[95314.520,93616.770,94962.390,94982.910,95099.810,95069.550,94447.610,94944.700,94537.780,94491.450,...,94080.290,94013.190,93619.640,94606.830,94017.780,94710.720,94789.720,94603.250,92354.950,94387.940],[94579.910,94161.630,94926.520,94345.330,94728.150,94314.710,94060.580,94687.140,94484.210,94554.700,...,94830.450,94992.480,94288.730,94386.070,94268.610,94194.480,94351.780,94721.170,95154.630,94946.030],[92466.090,93474.060,94623.550,94871.370,94543.730,94469.890,94177.810,94277.300,95093.750,95168.940,...,94812.660,94855.430,94745.950,94989.070,94344.100,94466.340,95031.980,94189.050,94135.820,94639.090],[94923.840,94527.370,94763.440,94332.220,94986.220,94403.270,94580.310,94827.420,94644.920,94394.050,...,94933.880,94685.200,94850.470,94823.960,94714.590,94536.290,94586.980,95094.750,94361.580,94603.430],[94778.950,94356.420,93509.130,94900.640,95024.140,92471.670,94256.250,94998.650,94863.750,95223.620,...,95586.390,951

In [ ]:
table.select(['pressure']).slice(5)

pyarrow.Table
pressure: double
----
pressure: [[95314.52,93616.77,94962.39,94982.91,95099.81,95069.55,94447.61,94944.7,94537.78,94491.45,...,94080.29,94013.19,93619.64,94606.83,94017.78,94710.72,94789.72,94603.25,92354.95,94387.94],[94579.91,94161.63,94926.52,94345.33,94728.15,94314.71,94060.58,94687.14,94484.21,94554.7,...,94830.45,94992.48,94288.73,94386.07,94268.61,94194.48,94351.78,94721.17,95154.63,94946.03],[92466.09,93474.06,94623.55,94871.37,94543.73,94469.89,94177.81,94277.3,95093.75,95168.94,...,94812.66,94855.43,94745.95,94989.07,94344.1,94466.34,95031.98,94189.05,94135.82,94639.09],[94923.84,94527.37,94763.44,94332.22,94986.22,94403.27,94580.31,94827.42,94644.92,94394.05,...,94933.88,94685.2,94850.47,94823.96,94714.59,94536.29,94586.98,95094.75,94361.58,94603.43],[94778.95,94356.42,93509.13,94900.64,95024.14,92471.67,94256.25,94998.65,94863.75,95223.62,...,95586.39,95153.58,94321.16,95819.33,94930.94,95410.19,94058.92,95174.34,95550.18,95468.33],[95239.17,95493.98,95405.7,9